In [28]:
import nltk
from nltk.corpus import udhr
import keras
from nltk import FreqDist
from gensim.models.word2vec import Word2Vec
import pandas as pd
import regex
import nltk.tokenize.casual
import tensorflow as tf

In [29]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time
import gc

import numpy as np
import matplotlib.pyplot as plt

In [30]:
# https://stackoverflow.com/questions/39263929/how-can-i-read-tar-gz-file-using-pandas-read-csv-with-gzip-compression-option
df = pd.read_csv('books_labels.tar.gz', compression='gzip', header=0, sep=',', quotechar='"', on_bad_lines='skip')

In [31]:
# https://stackoverflow.com/questions/17424182/extracting-all-rows-from-pandas-dataframe-that-have-certain-value-in-a-specific
spanish_df = df[df['language'] == 'SPA']['text']
spanish_list = spanish_df.to_list() # https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://ioflood.com/blog/dataframe-to-list-pandas/%23:~:text%3DIt%2520is%2520utilized%2520with%2520the,tolist()%2520.%26text%3DIn%2520the%2520example%2520above%252C%2520we,convert%2520it%2520into%2520a%2520list.&ved=2ahUKEwiP3deMnLSFAxVD4ckDHUo7BsEQFnoECA4QAw&usg=AOvVaw3GgfIVdsVo9Dxul02uata1

In [32]:
spanish_list_clean = [regex.sub(r'\([a-zA-z0-9]\)', '', item) for item in spanish_list]
spanish_string = ''.join(spanish_list_clean)
spanish_list_clean = spanish_string.split('.')
spanish_list_clean = [item.strip(' ') for item in spanish_list_clean]
spanish_corpus = [nltk.tokenize.casual_tokenize(item) for item in spanish_list_clean][:1000]
print(spanish_corpus[0])

['Estas', 'son', 'las', 'palabras', 'de', 'Amós', ',', 'que', 'era', 'un', 'pastor', 'de', 'Tecoa']


In [33]:
# https://stackoverflow.com/questions/17424182/extracting-all-rows-from-pandas-dataframe-that-have-certain-value-in-a-specific
portuguese_df = df[df['language'] == 'POR']['text']
portuguese_list = portuguese_df.to_list()

In [34]:
# portuguese_list_clean = [regex.sub(r'\([a-zA-z0-9]\)', '', item) for item in portuguese_list]
# print(portuguese_list_clean[:50])
for i, item in enumerate(portuguese_list):
    if type(item) == float:
        portuguese_list.pop(i)
portuguese_string = ''.join(portuguese_list)
portuguese_list = portuguese_string.split('.')
portuguese_list = [item.strip(' ') for item in portuguese_list]
portuguese_corpus = [nltk.tokenize.casual_tokenize(item) for item in portuguese_list][:1000]
print(portuguese_corpus[0])

['No', 'segundo', 'ano', 'do', 'rei', 'Dario', ',', 'no', 'sexto', 'mês', ',', 'no', 'primeiro', 'dia', 'do', 'mês', ',', 'veio', 'a', 'palavra', 'do', 'Senhor', ',', 'por', 'intermédio', 'do', 'profeta', 'Ageu', ',', 'a', 'Zorobabel', ',', 'governador', 'de', 'Judá', ',', 'filho', 'de', 'Sealtiel', ',', 'e', 'a', 'Josué', ',', 'o', 'sumo', 'sacerdote', ',', 'filho', 'de', 'Jeozadaque', ',', 'dizendo', ':', 'Assim', 'fala', 'o', 'Senhor', 'dos', 'exércitos', ',', 'dizendo', ':', 'Este', 'povo', 'diz', ':', 'Não', 'veio', 'ainda', 'o', 'tempo', ',', 'o', 'tempo', 'de', 'se', 'edificar', 'a', 'casa', 'do', 'Senhor']


In [35]:
del df
del spanish_string, portuguese_string, spanish_list, spanish_list_clean, portuguese_list
gc.collect()

0

In [40]:
# New Dataset: https://huggingface.co/datasets/versae/bibles
spanish_input_texts, spanish_target_texts = [], []
portuguese_input_texts, portuguese_target_texts = [], []
spanish_vocabulary = set()
portuguese_vocabulary = set()
start_token = '[START]'
stop_token = '[END]'
unknown_token = '[UNK]'
spanish_vocabulary.add(start_token)
spanish_vocabulary.add(stop_token)
spanish_vocabulary.add(unknown_token)
portuguese_vocabulary.add(start_token)
portuguese_vocabulary.add(stop_token)
portuguese_vocabulary.add(unknown_token)

# max_training_samples = min(25000, len(spanish_corpus) - 1, len(portuguese_corpus) - 1)

for spanish_input_text in spanish_corpus:
    spanish_target_text = [start_token] + spanish_input_text + [stop_token]
    spanish_input_texts.append(spanish_input_text)
    spanish_target_texts.append(spanish_target_text)
    for char in spanish_target_text:
        if char not in spanish_vocabulary:
            spanish_vocabulary.add(char)

for portuguese_input_text in portuguese_corpus:
    portuguese_target_text = [start_token] + portuguese_input_text + [stop_token]
    portuguese_input_texts.append(portuguese_input_text)
    portuguese_target_texts.append(portuguese_target_text)
    for char in portuguese_target_text:
        if char not in portuguese_vocabulary:
            portuguese_vocabulary.add(char)

unified_vocabulary = spanish_vocabulary.union(portuguese_vocabulary)

print(len(spanish_vocabulary), len(unified_vocabulary), len(portuguese_vocabulary))

3312 7304 4441


In [233]:
spanish_vocabulary = sorted(spanish_vocabulary)
portuguese_vocabulary = sorted(portuguese_vocabulary)

spanish_vocab_size = len(spanish_vocabulary)
portuguese_vocab_size = len(portuguese_vocabulary)
unified_vocab_size = len(unified_vocabulary)
max_spanish_seq_length = max([len(txt) for txt in spanish_target_texts])
max_portuguese_seq_length = max([len(txt) for txt in portuguese_target_texts])
max_unified_seq_length = max(max_spanish_seq_length, max_portuguese_seq_length)

spanish_token_index = dict([(token, i+1) for i, token in
                          enumerate(spanish_vocabulary)])
portuguese_token_index = dict([(token, i+1) for i, token in
                          enumerate(portuguese_vocabulary)])
unified_token_index = dict([(token, i+1) for i, token in
                          enumerate(unified_vocabulary)])
reverse_spanish_token_index = dict([(i, token) for token, i in
                          spanish_token_index.items()])
reverse_portuguese_token_index = dict([(i, token) for token, i in
                          portuguese_token_index.items()])
reverse_unified_token_index = dict([(i, token) for token, i in
                          unified_token_index.items()])

In [115]:
import numpy as np

# encoder_input_data = np.zeros((len(spanish_input_texts) + len(portuguese_input_texts),
#                                max(max_spanish_seq_length, max_portuguese_seq_length), spanish_vocab_size + portuguese_vocab_size),
#                                dtype='float32')

# Update with both
spanish_encoder_input_data = np.zeros((len(spanish_input_texts), max_spanish_seq_length),
                               dtype='int32')
spanish_decoder_input_data = np.zeros((len(spanish_input_texts), max_spanish_seq_length),
                               dtype='int32')
spanish_decoder_target_data = np.zeros((len(spanish_input_texts), max_spanish_seq_length),
                               dtype='int32')

portuguese_encoder_input_data = np.zeros((len(portuguese_input_texts), max_portuguese_seq_length),
                               dtype='int32')
portuguese_decoder_input_data = np.zeros((len(portuguese_input_texts), max_portuguese_seq_length),
                               dtype='int32')
portuguese_decoder_target_data = np.zeros((len(portuguese_input_texts), max_portuguese_seq_length),
                               dtype='int32')

for i, (input_text, target_text) in enumerate(zip(spanish_input_texts, spanish_target_texts)):
    for t, token in enumerate(input_text):
        spanish_encoder_input_data[
            i, t] = unified_token_index[token]
    for t, token in enumerate(target_text):
        spanish_decoder_input_data[
            i, t] = unified_token_index[token]
        if t > 0:
            spanish_decoder_target_data[i, t - 1] = spanish_token_index[token]

for i, (input_text, target_text) in enumerate(zip(portuguese_input_texts, portuguese_target_texts)):
    for t, token in enumerate(input_text):
        portuguese_encoder_input_data[
            i, t] = unified_token_index[token]
    for t, token in enumerate(target_text):
        portuguese_decoder_input_data[
            i, t] = unified_token_index[token]
        if t > 0:
            portuguese_decoder_target_data[i, t - 1] = portuguese_token_index[token]

In [116]:
spanish_encoder_input_data

array([[6050, 6841, 3449, ...,    0,    0,    0],
       [1182, 4266, 3804, ...,    0,    0,    0],
       [2371, 5567, 6348, ...,    0,    0,    0],
       ...,
       [2730, 5074, 1207, ...,    0,    0,    0],
       [6654,  409, 3449, ...,    0,    0,    0],
       [5073, 2529, 7296, ...,    0,    0,    0]], dtype=int32)

In [117]:
batch_size = 64
epochs = 20
num_neurons = 256

In [118]:
# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, LSTM, Dense


# spanish_encoder_inputs = Input(shape=(None, spanish_vocab_size))
# spanish_encoder = LSTM(num_neurons, return_state=True)
# spanish_encoder_outputs, s_state_h, s_state_c = spanish_encoder(spanish_encoder_inputs)
# spanish_encoder_states = (s_state_h, s_state_c)

# spanish_decoder_inputs = Input(shape=(None, spanish_vocab_size))
# spanish_decoder_lstm = LSTM(
#     num_neurons, return_sequences=True, return_state=True)
# spanish_decoder_outputs, _, _ = spanish_decoder_lstm(
#     spanish_decoder_inputs, initial_state=spanish_encoder_states)
# spanish_decoder_dense = Dense(
#     spanish_vocab_size, activation='softmax')
# spanish_decoder_outputs = spanish_decoder_dense(spanish_decoder_outputs)
# spanish_spanish_model = Model(
#     inputs=[spanish_encoder_inputs, spanish_decoder_inputs],
#     outputs=spanish_decoder_outputs)

# spanish_spanish_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
#               metrics=['acc'])
# spanish_spanish_model.fit([spanish_encoder_input_data, spanish_decoder_input_data],
#           spanish_decoder_target_data, batch_size=batch_size, epochs=epochs,
#           validation_split=0.1)

In [119]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [120]:
from transformers import *
# transformer = Transformer(
#     num_layers=num_layers,
#     d_model=d_model,
#     num_heads=num_heads,
#     dff=dff,
#     input_vocab_size=spanish_vocab_size,
#     target_vocab_size=spanish_vocab_size,
#     dropout_rate=dropout_rate)

In [121]:
encoder = Encoder(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    vocab_size=unified_vocab_size,
    dropout_rate=dropout_rate)

spanish_decoder = Decoder(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    vocab_size=unified_vocab_size,
    dropout_rate=dropout_rate)

portuguese_decoder = Decoder(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    vocab_size=unified_vocab_size,
    dropout_rate=dropout_rate)

In [122]:
spanish_transformer = ComposedTransformer(encoder, spanish_decoder, spanish_vocab_size)
portuguese_transformer = ComposedTransformer(encoder, portuguese_decoder, portuguese_vocab_size)

In [123]:
#output = transformer((spanish_encoder_input_data, spanish_decoder_target_data))
#print(output.shape)

In [124]:
learning_rate = CustomSchedule(d_model)

spanish_optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)
portuguese_optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [125]:
spanish_transformer.compile(
    loss=masked_loss,
    optimizer=spanish_optimizer,
    metrics=[masked_accuracy])
portuguese_transformer.compile(
    loss=masked_loss,
    optimizer=portuguese_optimizer,
    metrics=[masked_accuracy])

In [126]:
spanish_dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data, spanish_decoder_input_data), spanish_decoder_target_data))
portuguese_dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data, spanish_decoder_input_data), spanish_decoder_target_data))
spanish_batched_dataset = spanish_dataset.batch(batch_size)
portuguese_batched_dataset = portuguese_dataset.batch(batch_size)
#tf.data.Dataset.bucket_by_sequence_length(ds)

In [127]:
# attn_scores = transformer.decoder.dec_layers[-1].last_attn_scores
# print(attn_scores.shape)  # (batch, heads, target_seq, input_seq)

In [128]:
spanish_transformer.summary()

Model: "composed_transformer_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_17 (Encoder)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_3 (Decoder)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_191 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [130]:
tf.config.run_functions_eagerly(True)
for i in range(epochs):
    spanish_transformer.fit(spanish_batched_dataset)
    portuguese_transformer.fit(portuguese_batched_dataset)

16/16 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - loss: 5.9730 - masked_accuracy: 0.1348
16/16 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - loss: 6.1436 - masked_accuracy: 0.1447
16/16 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - loss: 5.8560 - masked_accuracy: 0.1503
16/16 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - loss: 6.0232 - masked_accuracy: 0.1583
16/16 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - loss: 5.7411 - masked_accuracy: 0.1712
16/16 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - loss: 5.9047 - masked_accuracy: 0.1722
16/16 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - loss: 5.6295 - masked_accuracy: 0.1909
16/16 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - loss: 5.7903 - masked_accuracy: 0.1857
16/16 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - loss: 5.5170 - masked_accuracy: 0.2100
16/16 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - loss: 5.6807 - masked_accuracy: 0.1953
16/16 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - loss: 5.4081 - masked_accuracy: 0.2217
16/16 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - loss: 5.5742 - masked_accuracy: 0.2122
16/16 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step -

In [234]:
class Translator(tf.Module):
  def __init__(self, transformer, input_token_index, reverse_input_token_index, output_token_index, reverse_output_token_index):
    self.transformer = transformer
    self.input_token_index = input_token_index
    self.output_token_index = output_token_index
    self.reverse_input_token_index = reverse_input_token_index
    self.reverse_output_token_index = reverse_output_token_index

  def __call__(self, sentence, max_length):
    # The input sentence is Portuguese, hence adding the `[START]` and `[END]` tokens.
    # assert isinstance(sentence, tf.Tensor)
    # if len(sentence.shape) == 0:
    #   sentence = sentence[tf.newaxis]
    tokenized_sentence = nltk.tokenize.casual_tokenize(sentence)
    sentence_tensor = np.zeros(max_length, dtype='int64')
    for t, token in enumerate(tokenized_sentence):
      sentence_tensor[t] = self.input_token_index[token]
    #sentence = self.tokenizers.pt.tokenize(sentence).to_tensor()

    encoder_input = sentence_tensor[tf.newaxis]
    
    # As the output language is English, initialize the output with the
    # English `[START]` token.
    #start_end = self.tokenizers.en.tokenize([''])[0]
    start = self.input_token_index['[START]']
    end = self.output_token_index['[END]']

    # `tf.TensorArray` is required here (instead of a Python list), so that the
    # dynamic-loop can be traced by `tf.function`.
    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)
    for i in tf.range(max_length):
      output = tf.transpose(output_array.stack())[tf.newaxis]
      predictions = self.transformer([encoder_input, output], training=False)

      # Select the last token from the `seq_len` dimension.
      predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.

      predicted_id = tf.argmax(predictions, axis=-1)

      # Concatenate the `predicted_id` to the output which is given to the
      # decoder as its input.
      output_array = output_array.write(i+1, predicted_id[0][0])

      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())
    # The output shape is `(1, tokens)`. https://www.tensorflow.org/api_docs/python/tf/cast
    text = [item.numpy() for item in output]  # Shape: `()`.

    # `tf.function` prevents us from using the attention_weights that were
    # calculated on the last iteration of the loop.
    # So, recalculate them outside the loop.
    # self.transformer([encoder_input, output[:,:-1]], training=False)
    # attention_weights = self.transformer.decoder.last_attn_scores

    return text #, attention_weights

In [255]:
sentence = 'Estas son las palabras de Amós , que era un pastor de Tecoa'
spanish_translator = Translator(spanish_transformer, unified_token_index, reverse_unified_token_index, portuguese_token_index, reverse_portuguese_token_index)
translated_text = spanish_translator(
    sentence,15)

/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer doe

In [260]:
print(translated_text)

[5690, 630, 1271, 2642, 1271, 2642, 1271, 2642, 1271, 2642, 1271, 2642, 1271, 1271, 1271, 1271]


In [261]:
for i in range(20):
    try:
        print(reverse_spanish_token_index[translated_text[i]])
    except Exception:
        continue

Y
de
que
de
que
de
que
de
que
de
que
de
de
de
de
